# Pré-Processamento dos Documentos

A limpeza dos dados é um processo essencial para garantir a qualidade e a confiabilidade das informações armazenadas em um banco de dados. A limpeza dos dados envolve a identificação e a correção de erros, inconsistências, duplicidades e valores ausentes nos dados. A arquitetura do armazenamento é a forma como os dados são organizados, estruturados e acessados em um banco de dados. Uma das opções de arquitetura é o formato YAML, que significa YAML Ain't Markup Language. O YAML é um formato de serialização de dados que usa uma sintaxe simples e legível para representar estruturas de dados como listas, mapas, sequências e escalares. O YAML é compatível com diversas linguagens de programação e pode ser usado para armazenar dados de forma hierárquica e flexível.

<!-- <hr style="border-width: 1px" width="95%" > -->
<div></div> 

In [9]:
# Importa os módulos necessários
import os    # Módulo para lidar com funções do sistema operacional
import gc    # Módulo para realizar coleta de lixo e gerenciamento de memória
import sys

import numpy as np   # Módulo para trabalhar com matrizes e funções matemáticas
import pandas as pd  # Módulo para trabalhar com dataframes e séries em Python


<div></div> 

## Estruturação dos Arquivos

<div></div> 

In [10]:
disney = pd.read_csv('../data/movies/disney_titles.csv')
prime = pd.read_csv('../data/movies/prime_titles.csv')
hbo = pd.read_csv('../data/movies/hbo_titles.csv')
netflix = pd.read_csv('../data/movies/netflix_titles.csv')

disney['provider'] = 'disney'
prime['provider'] = 'prime'
hbo['provider'] = 'hbo'
netflix['provider'] = 'netflix'

base = pd.concat([disney, prime, hbo, netflix])

del disney, prime, hbo, netflix

base.dropna(ignore_index=True, inplace=True)
base.reset_index(drop=True, inplace=True)


<div></div> 

## Processamento de Texto

<div></div> 


### Tratamento dos gêneros

In [11]:
from ast import literal_eval

base['genres'] = base['genres'].fillna('[]').apply(literal_eval).apply(lambda x: [i for i in x] if isinstance(x, list) else [])


### Transformação de minúsculos

<div></div> 

In [12]:
# (\[a-z]): para encontrar todos os caracteres que começam com uma barra invertida () seguida por uma letra minúscula (a-z);
# ([^\w\]): para encontrar todos os caracteres que não são letras, números ou barras invertidas ();
# (\S+\d\S+): para encontrar todos os trechos de texto que contêm um ou mais caracteres não brancos (\S), 
# seguidos por um dígito (\d), seguidos por mais um ou mais caracteres não brancos (\S).
base['post'] = base['description'].replace(r'(\\[a-z])|([^\w\\])|(\S+\d\S+)', ' ', regex=True)

# Aplicando as funções str.lower() e str.strip() simultaneamente
base['post'] = base['post'].apply(lambda x: x.lower().strip() if isinstance(x, str) else x)


<div></div> 

### Tokenização e Lemmatizer

<div></div>

In [13]:
from ir.preprocessing import lemmatize_word

base['post'] = base['post'].apply(lambda x: ' '.join([lemmatize_word(word.lower()) for word in x.split()]))


### Identificação das query / docs

In [14]:
np.random.seed(42)
rand = np.random.random(base.shape[0])

d_index = rand <  0.7
q_index = rand >= 0.7

### TF IDF

In [15]:
from ir.tf_idf import tfidf

In [16]:
weights = tfidf(base, 'post').T

In [17]:
from sklearn.metrics.pairwise import linear_kernel

### Ranqueamento

In [18]:
q_index = base[q_index].index
d_index = base[d_index].index

In [23]:
rank_geral = linear_kernel(weights.iloc[d_index], weights.iloc[q_index])
rank_geral = pd.DataFrame(rank_geral, index=d_index, columns=q_index)

In [78]:
import sys
import numba

def calcular_resultados_relevantes(q_index: list, base: pd.DataFrame) -> 'resultados_relevantes[dict], resultados_sistema[dict]':
    resultados_sistema = {}

    for q in q_index: 
        resultados_sistema[q] = rank_geral[q].sort_values(ascending=False).index

    resultados_relevantes = {}

    for q in q_index:
        q_genre = base.iloc[q]['genres']

        k = []

        for d in resultados_sistema[q]:
            d_genre = base.iloc[d]['genres']
            
            # Verifica qual lista de gêneros é menor para otimizar a comparação
            if len(d_genre) > len(q_genre):
                comparativo_menor = q_genre
                comparativo_maior = d_genre
            else:
                comparativo_menor = d_genre
                comparativo_maior = q_genre
            
            # Verifica se há pelo menos um gênero em comum entre as listas
            partial_relevance = any(i in comparativo_maior for i in comparativo_menor)
            
            if partial_relevance:
                k.append(d)
        
        print(f'\rQuery: {q}/{q_index.max()} - Doc: {d}/{d_index.max()}', end='')
        sys.stdout.flush()

        resultados_relevantes[q] = k
        
    return resultados_relevantes, resultados_sistema

resultados_relevantes, resultados_sistema = calcular_resultados_relevantes(q_index, base)


Query: 3093/3093 - Doc: 1730/3092

         3 function calls in 0.000 seconds

   Ordered by: internal time

   ncalls  tottime  percall  cumtime  percall filename:lineno(function)
        1    0.000    0.000    0.000    0.000 {built-in method builtins.exec}
        1    0.000    0.000    0.000    0.000 <string>:1(<module>)
        1    0.000    0.000    0.000    0.000 {method 'disable' of '_lsprof.Profiler' objects}

### Métricas

In [89]:
def calcular_p_n_media(resultados_relevantes, resultados_sistema, n):
    def calcular_p_n(resultados, relevantes):
        if len(resultados) > n:
            resultados = resultados[:n]  # Considerar apenas os primeiros n resultados
        num_relevantes = len(set(resultados) & set(relevantes))  # Contar quantos resultados relevantes foram encontrados
        p_n = num_relevantes / n  # Calcular a precisão P@n
        return p_n

    p_n_total = 0
    for consulta, relevantes in resultados_relevantes.items():
        resultados = resultados_sistema.get(consulta, [])  # Obtém os resultados retornados pelo sistema para a consulta
        p_n = calcular_p_n(resultados, relevantes)
        p_n_total += p_n

    p_n_media = p_n_total / len(resultados_relevantes)
    return p_n_media

# Utilizando a função para calcular a média do P@n
p_n_media = calcular_p_n_media(resultados_relevantes, resultados_sistema, n=10)
print(f"Média do P@: {p_n_media}")

Média do P@: 0.719268030139934


In [85]:
for x in [10, 20, 50, 100]: 
    print(f"Média do P@{x}: {calcular_p_n_media(resultados_relevantes, resultados_sistema, n=x)}")

Média do P@10: 0.719268030139934
Média do P@20: 0.6967168998923577
Média do P@50: 0.6745748116254034
Média do P@100: 0.658902045209902


In [83]:
def average_precision(relevantes, recomendados):
    relevancia_cumulativa = 0
    precision_cumulativa = 0
    num_relevantes = len(relevantes)
    ap = 0

    for i, rec in enumerate(recomendados):
        if rec in relevantes:
            relevancia_cumulativa += 1
            precision_cumulativa += relevancia_cumulativa / (i + 1)

    if num_relevantes > 0:
        ap = precision_cumulativa / num_relevantes

    return ap

def mean_average_precision(resultados_relevantes, resultados_sistema):
    map = 0
    num_consultas = len(resultados_relevantes)

    for q in resultados_relevantes:
        relevantes = resultados_relevantes[q]
        recomendados = resultados_sistema[q]
        ap = average_precision(relevantes, recomendados)
        map += ap

    if num_consultas > 0:
        map /= num_consultas

    return map

# Aplicar o MAP nas consultas
map_result = mean_average_precision(resultados_relevantes, resultados_sistema)
map_result

0.5882758350175267